In [2]:
import os, sys
sys.path.append("motion_generation")
sys.path.append("rig_agnostic_encoding/functions")
sys.path.append("rig_agnostic_encoding/models")

from motion_generation.MoE import MoE
import motion_generation
from motion_generation.GRU import GRU
from motion_generation.LSTM import LSTM
from motion_generation.MotionGeneration import MotionGenerationModel
from motion_generation.MotionGeneration_v2 import MotionGenerationModel as MotionGenerationModel_v2
from motion_generation.MotionGeneration_v3 import MotionGenerationModel as MotionGenerationModel_v3
from motion_generation.MotionGenerationRNN import MotionGenerationModelRNN
from motion_generation.MotionGenerationBatch import MotionGenerationModelBatch
from rig_agnostic_encoding.models.MLP import MLP
from rig_agnostic_encoding.models.MLP_MIX import MLP_MIX
from rig_agnostic_encoding.models.VAE import VAE
from rig_agnostic_encoding.functions.DataProcessingFunctions import clean_checkpoints
from GlobalSettings import MODEL_PATH
import bz2
from cytoolz import concat
import torch
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
import func as F
import _pickle as pickle
import json as js
import importlib
import random

In [19]:
config = {
    "hidden_dim": 512,
    "k": 512,
    "lr": 1e-4,
    "batch_size": 32,
    "keep_prob": .2,
    "loss_fn":torch.nn.functional.mse_loss,
    "optimizer":torch.optim.AdamW,
    "scheduler":torch.optim.lr_scheduler.StepLR,
    "scheduler_param": {"step_size":80, "gamma":.9},
    "basis_func":"gaussian",
    "n_centroid":128,
    "k_experts": 4,
    "gate_size": 128,
    "g_hidden_dim": 512,
    "num_layers": 4,
    "autoregress_prob":0,
    "autoregress_inc":.2,
    "autoregress_ep":50,
    "autoregress_max_prob":.5,
    "cost_hidden_dim":128,
    "seq_len":13,
    "device":"cuda"
    }

In [7]:
def getFilesNames(file_paths, data_path, MAX_FILES=-1):
    for dname, dirs, files in os.walk(data_path):
        for i, file in enumerate(files):
            file_paths.append(os.path.join(dname, file))
            if MAX_FILES > 0 and i >= MAX_FILES:
                break
    return file_paths
data_path = "/home/nuoc/Documents/MEX/data/data/Dataset_R1_Two_1"
data_path2 = "/home/nuoc/Documents/MEX/data/data/Dataset_R2_Two_1"
data_path3 = "/home/nuoc/Documents/MEX/data/data/Dataset_R3_Two_1"
data_path4 = "/home/nuoc/Documents/MEX/data/data/Dataset_R4_Two_1"
file_paths = getFilesNames([],data_path)
file_paths2 = getFilesNames([],data_path2)
file_paths3 = getFilesNames([],data_path3)
file_paths4 = getFilesNames([],data_path4)

print(len(file_paths))
print(len(file_paths2))





240
240


In [9]:
phase_features = ["phase_vec_l2"]
pose_features = ["pos", "rotMat2", "velocity"]
cost_features = ["posCost", "rotCost"]
target_features = ["targetPosition", "targetRotation"]
features = phase_features + pose_features + cost_features + target_features
clips = []
feature_dims = {}


In [10]:
data = F.process_data_multithread(file_paths, features)
data2 = F.process_data_multithread(file_paths2, features)
data3 = F.process_data_multithread(file_paths3, features)
data4 = F.process_data_multithread(file_paths4, features)

2021-05-07 21:48:41,987	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265
2021-05-07 21:49:01,252	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265
2021-05-07 21:49:21,652	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265
2021-05-07 21:49:40,748	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265


In [14]:
feature_dims = data[0][1]
feature_dims2 = data2[0][1]
feature_dims3 = data3[0][1]
feature_dims4 = data4[0][1]

clips = [np.copy(d[0]) for d in data]
clips2 = [np.copy(d[0]) for d in data2]
clips3 = [np.copy(d[0]) for d in data3]
clips4 = [np.copy(d[0]) for d in data4]



In [15]:
phase_dim = sum([feature_dims[feature] for feature in phase_features])
pose_dim = sum([feature_dims[feature] for feature in pose_features])
pose_dim2 = sum([feature_dims2[feature] for feature in pose_features])
pose_dim3 = sum([feature_dims3[feature] for feature in pose_features])
pose_dim4 = sum([feature_dims4[feature] for feature in pose_features])
cost_dim = sum([feature_dims[feature] for feature in cost_features])
target_dim = sum([feature_dims[feature] for feature in target_features])
print(phase_dim, " ", cost_dim, " ", target_dim)
print(pose_dim, pose_dim2, pose_dim3, pose_dim4)

8   24   48
372 372 348 288


In [17]:
x_tensors = torch.stack([F.normaliseT(torch.from_numpy(clip[:-1])).float() for clip in clips])
y_tensors = torch.stack([torch.from_numpy(clip[1:]).float() for clip in clips])

x_tensors2 = torch.stack([F.normaliseT(torch.from_numpy(clip[:-1])).float() for clip in clips2])
y_tensors2 = torch.stack([torch.from_numpy(clip[1:]).float() for clip in clips2])

x_tensors3 = torch.stack([F.normaliseT(torch.from_numpy(clip[:-1])).float() for clip in clips3])
y_tensors3 = torch.stack([torch.from_numpy(clip[1:]).float() for clip in clips3])

x_tensors4 = torch.stack([F.normaliseT(torch.from_numpy(clip[:-1])).float() for clip in clips4])
y_tensors4 = torch.stack([torch.from_numpy(clip[1:]).float() for clip in clips4])

pose_data1 = x_tensors[:,  :,  phase_dim:phase_dim+pose_dim]
pose_data2 = x_tensors2[:, :, phase_dim:phase_dim+pose_dim2]
pose_data3 = x_tensors3[:, :, phase_dim:phase_dim+pose_dim3]
pose_data4 = x_tensors4[:, :, phase_dim:phase_dim+pose_dim4]
pose_data = torch.cat((pose_data1, pose_data2, pose_data3, pose_data4), dim=2)

dataset = TensorDataset(pose_data, pose_data)
datasetR1 = TensorDataset(x_tensors, y_tensors)
datasetR2 = TensorDataset(x_tensors2, y_tensors2)
datasetR3 = TensorDataset(x_tensors3, y_tensors3)
datasetR4 = TensorDataset(x_tensors4, y_tensors4)
datasetR1_R3 = TensorDataset(x_tensors, y_tensors3)

N = len(x_tensors)

train_ratio = int(.7*N)
val_ratio = int((N-train_ratio) / 2.0)
test_ratio = N - train_ratio - val_ratio
train_set, val_set, test_set = random_split(dataset, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
train_setR1, val_setR1, test_setR1 = random_split(datasetR1, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
train_setR2, val_setR2, test_setR2 = random_split(datasetR2, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
train_setR3, val_setR3, test_setR3 = random_split(datasetR3, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
train_setR4, val_setR4, test_setR4 = random_split(datasetR4, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
train_setR1_3, val_setR1_3, test_setR1_3 = random_split(datasetR1_R3, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
# print(len(train_set), len(val_set), len(test_set))
# print(len(train_set2), len(val_set2), len(test_set2))

In [38]:
def extract_targets(train_set, val_set, test_set, target_dim):
    t1, t2, t3 = [], [], []
    for i in range(len(train_set)):
        x = train_set[i][0]
        y = train_set[i][1]
        t1.append((x[:, :-target_dim], y[:, :-target_dim]))
    for i in range(len(val_set)):
        x = val_set[i][0]
        y = val_set[i][1]
        t2.append((x[:, :-target_dim:], y[:, :-target_dim:]))
    for i in range(len(test_set)):
        x = test_set[i][0]
        y = test_set[i][1]
        t3.append((x[:, :-target_dim:], y))
    return t1, t2, t3



In [48]:
t1, v1, te1 = extract_targets(train_set=train_setR1, val_set=val_setR1, test_set=test_setR1, target_dim=target_dim)
t2, v2, te2 = extract_targets(train_set=train_setR2, val_set=val_setR2, test_set=test_setR2, target_dim=target_dim)
t3, v3, te3 = extract_targets(train_set=train_setR3, val_set=val_setR3, test_set=test_setR3, target_dim=target_dim)
t4, v4, te4 = extract_targets(train_set=train_setR4, val_set=val_setR4, test_set=test_setR4, target_dim=target_dim)
t1_3, v1_3, te1_3 = extract_targets(train_set=train_setR1_3, val_set=val_setR1_3, test_set=test_setR1_3, target_dim=target_dim)

In [ ]:
filename = "/home/nuoc/Documents/MEX/models/version_0.2/MLP_MoE_R1_FULL/0.018650446087121964.pbz2"

with bz2.BZ2File(filename, "rb") as f:
    obj = pickle.load(f)

pose_autoencoder = MLP.load_checkpoint(obj["pose_autoencoder_path"])
cost_encoder = MLP.load_checkpoint(obj["cost_encoder_path"])
generationModel = MoE.load_checkpoint(obj["motionGenerationModelPath"])

model = MotionGenerationModel(config=obj["config"], feature_dims=obj["feature_dims"], Model=MoE,
                              input_slicers=obj["in_slices"], output_slicers=obj["out_slices"],
                              name=obj["name"])
# if MiddleModel is None:
MiddleModel = torch.nn.Sequential(
    torch.nn.Linear(in_features=pose_autoencoder.dimensions[-1], out_features=pose_autoencoder.dimensions[-1]))

MiddleModel.load_state_dict(obj["middle_layer_dict"])
model.in_slices = obj["in_slices"]
model.out_slices = obj["out_slices"]
model.pose_autoencoder = pose_autoencoder
model.cost_encoder = cost_encoder
model.generationModel = generationModel

In [65]:
model.train_set = train_set
model.val_set = val_set
model.test_set = test_set
model.config = config
model.cuda()

MotionGenerationModel(
  (pose_autoencoder): MLP(
    (encoder): Sequential(
      (fc0): Linear(in_features=372, out_features=512, bias=True)
      (act0): ELU(alpha=1.0)
      (drop): Dropout(p=0.2, inplace=False)
      (fc1): Linear(in_features=512, out_features=512, bias=True)
      (act1): ELU(alpha=1.0)
      (fc2): Linear(in_features=512, out_features=256, bias=True)
    )
    (decoder): Sequential(
      (fc0): Linear(in_features=256, out_features=512, bias=True)
      (act0): ELU(alpha=1.0)
      (drop): Dropout(p=0.2, inplace=False)
      (fc1): Linear(in_features=512, out_features=512, bias=True)
      (act1): ELU(alpha=1.0)
      (fc2): Linear(in_features=512, out_features=372, bias=True)
    )
  )
  (middle_layer): Linear(in_features=256, out_features=256, bias=True)
  (cost_encoder): MLP(
    (encoder): Sequential(
      (fc0): Linear(in_features=24, out_features=128, bias=True)
      (act0): ELU(alpha=1.0)
      (drop): Dropout(p=0.2, inplace=False)
      (fc1): Linear(i

In [20]:
Mixer_name = "MLPMIX_R1-R4"
MLPMIX = MLP_MIX(config=config, input_dims=[pose_dim, pose_dim2, pose_dim3, pose_dim4],
                 train_set=train_set, val_set=val_set, test_set=test_set,
                 name=Mixer_name)


In [21]:
MAX_EPOCHS = 200

checkpoint_callback = ModelCheckpoint(monitor="avg_val_loss", save_top_k=3)
earlystopping = EarlyStopping(monitor="avg_val_loss", patience=10)
logger=TensorBoardLogger(save_dir="logs/", name=Mixer_name, version="0.4")

trainer = pl.Trainer(
    default_root_dir="/home/nuoc/Documents/MEX/src/motion_generation/checkpoints",
    gpus=1, precision=16,
    callbacks=[earlystopping],
    min_epochs=20,
    logger=logger,
    max_epochs=MAX_EPOCHS,
    stochastic_weight_avg=True
)


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.


In [22]:
trainer.fit(MLPMIX)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type       | Params
---------------------------------------------
0 | cluster_model | Sequential | 262 K 
---------------------------------------------
262 K     Trainable params
0         Non-trainable params
262 K     Total params
1.051     Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7f7ac1512cd0> for <torch.optim.swa_utils.SWALR object at 0x7f7ac1521100>
  warnings.warn(*args, **kwargs)


1

In [23]:
trainer.test(MLPMIX)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.5591012239456177, 'test_loss': 0.5810086727142334}
--------------------------------------------------------------------------------


[{'test_loss': 0.5810086727142334, 'ptl/test_loss': 0.5591012239456177}]

In [169]:
# model_name = "MLP_MoE_R1_Two"
# model_name = "MLP_MoE_R2_Two"
# model_name = "MLP_MoE_R3_Two"
model_name = "MLP_MoE_R4_Two"

featureDim = {
    "phase_dim": phase_dim,
    # "pose_dim": pose_dim,
    # "pose_dim": pose_dim2,
    # "pose_dim": pose_dim3,
    "pose_dim": pose_dim4,
    "cost_dim": cost_dim,
    "g_input_dim": config["k"] + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim
    }

# in_slice = [phase_dim, pose_dim, cost_dim]
# in_slice = [phase_dim, pose_dim2, cost_dim]
# in_slice = [phase_dim, pose_dim3, cost_dim]
in_slice = [phase_dim, pose_dim4, cost_dim]
out_slice = [phase_dim, config["k"], cost_dim]

temp = MLP_MIX(config=config, input_dims=[pose_dim,pose_dim2, pose_dim3, pose_dim4])
# pose_encoder = temp.active_models[0]
# pose_encoder = temp.active_models[1]
# pose_encoder = temp.active_models[2]
pose_encoder = temp.active_models[3]
# pose_encoder.encoder.load_state_dict(MLPMIX.active_models[0].encoder.state_dict())
# pose_encoder.decoder.load_state_dict(MLPMIX.active_models[0].decoder.state_dict())

# pose_encoder.encoder.load_state_dict(MLPMIX.active_models[1].encoder.state_dict())
# pose_encoder.decoder.load_state_dict(MLPMIX.active_models[1].decoder.state_dict())

# pose_encoder.encoder.load_state_dict(MLPMIX.active_models[2].encoder.state_dict())
# pose_encoder.decoder.load_state_dict(MLPMIX.active_models[2].decoder.state_dict())

pose_encoder.encoder.load_state_dict(MLPMIX.active_models[3].encoder.state_dict())
pose_encoder.decoder.load_state_dict(MLPMIX.active_models[3].decoder.state_dict())
# middle_layer = torch.nn.Sequential()
middle_layer = temp.cluster_model
middle_layer.load_state_dict(MLPMIX.cluster_model.state_dict())
# cost_encoder = model2.cost_encoder

# generation_model = model2.generationModel
model = MotionGenerationModel_v2(config=config, Model=MoE, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim,
                                 input_slicers=in_slice, output_slicers=out_slice,
                                 # train_set=t1, val_set=v1, test_set=te1,
                                 # train_set=t2, val_set=v2, test_set=te2,
                                 # train_set=t3, val_set=v3, test_set=te3,
                                 train_set=t4, val_set=v4, test_set=te4,
                                 name=model_name
                                   )

In [61]:
model2 = MotionGenerationModel_v2.load_checkpoint(
    "/home/nuoc/Documents/MEX/models/version_0.2/MLP_MoE_R1_Two/0.0415918305516243.pbz2",
    Model=MoE, MiddleModel=middle_layer
)

In [170]:
model.generationModel.gate.load_state_dict(model2.generationModel.gate.state_dict())
model.generationModel.load_state_dict(model2.generationModel.state_dict())

model.generationModel.freeze()
model.cost_encoder.freeze()
# model.middle_layer.requires_grad_(False)

In [171]:

MAX_EPOCHS = 100

checkpoint_callback = ModelCheckpoint(monitor="avg_val_loss", save_top_k=3)
earlystopping = EarlyStopping(monitor="avg_val_loss", patience=20)
logger=TensorBoardLogger(save_dir="logs/", name=model_name, version="0.5")

trainer = pl.Trainer(
    default_root_dir="/home/nuoc/Documents/MEX/src/motion_generation/checkpoints",
    gpus=1, precision=16,
    callbacks=[earlystopping],
    min_epochs=20,
    logger=logger,
    max_epochs=MAX_EPOCHS,
    stochastic_weight_avg=True
)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.


In [172]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type       | Params
------------------------------------------------
0 | pose_autoencoder | MLP        | 1.3 M 
1 | middle_layer     | Sequential | 262 K 
2 | cost_encoder     | MLP        | 36.2 K
3 | generationModel  | MoE        | 3.5 M 
------------------------------------------------
1.6 M     Trainable params
3.5 M     Non-trainable params
5.1 M     Total params
20.572    Total estimated model params size (MB)


1

In [173]:
te = []
for i in range(len(test_setR4)):
    x = test_setR4[i][0]
    y = test_setR4[i][1]
    te.append((x[:, :-target_dim:], y[:, :-target_dim]))

trainer.test(model, test_dataloaders=DataLoader(te, pin_memory=True, num_workers=6))


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.055365629494190216, 'test_loss': 0.09592478722333908}
--------------------------------------------------------------------------------


[{'test_loss': 0.09592478722333908, 'ptl/test_loss': 0.055365629494190216}]

In [180]:
# te = te1
# te = te2
# te = te3
te = te4
n = 10
idx = np.random.randint(0, len(te), n)
original = []
generated = []
# pose_idx_upper = feature_dims2["phase_dim"] + feature_dims["pos"] + feature_dims["rotMat2"]
pose_idx_upper = model.in_slices[1] + feature_dims["pos"] + feature_dims["rotMat2"]
print(pose_idx_upper)


287


In [181]:
model.autoregress_prob = 1
with torch.no_grad():
    model.eval()
    model.cpu()
    # for i in range(1):
    # original =
    # x = x_tensors[idx]
    x = torch.stack([te[i][0] for i in idx])
    y = torch.stack([te[i][1] for i in idx])
    shape = x.shape
    x = x.view(-1, shape[-1])

    # x = x.view(-1, 13, shape[-1])
    # x_c = x[:,0,:]
    # n = shape[1]
    g_frames = []
    #
    # for i in range(0, 13):
    out= model(x)
    x_c = torch.cat(out,dim=1).detach()
    # g_frames.append(x_c.unsqueeze(1))

    # out = torch.cat(model(x), dim=1).view(shape)
            # x_c = torch.cat(out, dim=1)
    # g_frames.append(x_c.unsqueeze(1))
        # original.append(o_frames)
    # generated.append(torch.cat(g_frames, dim=1))
    generated = x_c
    # generated = torch.cat(g_frames, dim=1)
    generated = generated.view(shape)



In [182]:
print(y.size())
print(generated.size())

torch.Size([10, 299, 368])
torch.Size([10, 299, 320])


In [177]:
phase_dim = feature_dims["phase_vec_l2"]
toPosDim = phase_dim+feature_dims["pos"]
toRotDim = toPosDim + feature_dims["rotMat2"]
toVelDim = toRotDim + feature_dims["velocity"]

gPos = generated[:, :, phase_dim:toPosDim]
gRot = generated[:, :, toPosDim:toRotDim]
gVel = generated[:, :, toRotDim:toVelDim]

oPos = y[:, :, phase_dim:toPosDim]
oRot = y[:, :, toPosDim:toRotDim]
oVel = y[:, :, toRotDim:toVelDim]

tPos = y[:, :, -target_dim:-target_dim+3*4]
tRot = y[:, :, -target_dim+3*4:]

print(gPos.shape, gRot.shape, gVel.shape)
print(oPos.shape, oRot.shape, oVel.shape)
print(tPos.shape, tRot.shape)


torch.Size([10, 299, 93]) torch.Size([10, 299, 186]) torch.Size([10, 299, 33])
torch.Size([10, 299, 93]) torch.Size([10, 299, 186]) torch.Size([10, 299, 81])
torch.Size([10, 299, 12]) torch.Size([10, 299, 36])


In [178]:
clip_length = gPos.shape[1]
gPos_r = gPos.reshape((n, clip_length, -1, 3))
gRot_r = gRot.reshape((n, clip_length, -1, 3, 2))
gVel_r = gVel.reshape((n, clip_length, -1, 3))

oPos_r = oPos.reshape((n, clip_length, -1, 3))
oRot_r = oRot.reshape((n, clip_length, -1, 3, 2))
oVel_r = oVel.reshape((n, clip_length, -1, 3))

tPos_r = tPos.reshape((n, clip_length, -1, 3))
tRot_r = tRot.reshape((n, clip_length, -1, 3, 3))

print("Pos loss: ", torch.nn.functional.mse_loss(gPos_r, oPos_r))
print("Rot loss: ", torch.nn.functional.mse_loss(gRot_r, oRot_r))
print("Vel loss: ", torch.nn.functional.mse_loss(gVel_r, oVel_r))



Pos loss:  tensor(0.0176)
Rot loss:  tensor(0.0459)


<ipython-input-178-782b3a947c80>:15: UserWarning: Using a target size (torch.Size([10, 299, 27, 3])) that is different to the input size (torch.Size([10, 299, 11, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  print("Vel loss: ", torch.nn.functional.mse_loss(gVel_r, oVel_r))


RuntimeError: The size of tensor a (11) must match the size of tensor b (27) at non-singleton dimension 2

In [58]:
template = js.load(open("/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/TestAll_1_R1_One_1/False_2_0.json"))

In [179]:
def setVec3(struct, vec):
    struct["x"] = vec[0].item()
    struct["y"] = vec[1].item()
    struct["z"] = vec[2].item()

def setVec6(struct, vec):
    for r, cell in enumerate(["x", "y", "z"]):
        for col, column in enumerate(["c0", "c1"]):
            struct[column][cell] = vec[r, col].item()

def insert_pos(positions, rotations, velocity, tPos, tRot, name="Replay"):
    shape = positions.shape
    for c in range(shape[0]):
        for f in range(shape[1]):
            t = 0
            for j in range(shape[2]):
                jo = template["frames"][f]["joints"][j]
                setVec3(jo["position"], positions[c,f,j])
                setVec3(jo["velocity"], velocity[c,f,j])
                setVec6(jo["rotMat"], rotations[c,f,j])

                # jo["position"]["x"] = positions[c,f,j,0].item()
                # jo["position"]["y"] = positions[c,f,j,1].item()
                # jo["position"]["z"] = positions[c,f,j,2].item()

                # jo["velocity"]["x"] = velocity[c,f,j,0].item()
                # jo["velocity"]["y"] = velocity[c,f,j,1].item()
                # jo["velocity"]["z"<] = velocity[c,f,j,2].item()
                #
                # for r, cell in enumerate(["x", "y", "z"]):
                #     for col, column in enumerate(["c0", "c1"]):
                #         jo["rotMat"][column][cell] = rotations[c,f,j,r, col].item()

                if jo["key"]:
                    setVec3(jo["cost"]["TargetPosition"], tPos[c,f,t])
                    setVec6(jo["cost"]["TargetRotation"], tRot[c,f,t])
                    t+=1
                    # jo["targetPosition"]["x"] = tPos[c, f, t, 0]
                    # jo["targetPosition"]["y"] = tPos[c, f, t, 1]
                    # jo["targetPosition"]["z"] = tPos[c, f, t, 2]

                    # for r, cell in enumerate(["x", "y", "z"]):
                    #     for col, column in enumerate(["c0", "c1"]):
                    #         jo["rotMat"][column][cell] = rotations[c,f,j,r, col].item()
        with open("{}_{}.json".format(name, c), "w") as f:
            js.dump(template, f)

# os.mkdir("/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/R1_ALL")
insert_pos(oPos_r, oRot_r, oVel_r, tPos_r, tRot_r,
           "/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/R1_ALL/"+model_name+"_v5_Original")
insert_pos(gPos_r, gRot_r, oVel_r, tPos_r, tRot_r,
           "/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/R1_ALL/"+model_name+"_v5_Generated")


IndexError: index 27 is out of bounds for dimension 2 with size 27